Agora que entendemos o básico sobre CRS, podemos continuar estudando com mais tranquilidade. Nosso objetivo, nessa etapa, é equalizar os CRSs de dois arquivos com os quais iremos trabalhar.

Separamos, para essa etapa, as duas fontes de dados a seguir:

https://www.lapig.iesa.ufg.br/lapig/cursos_online/gvsig/a_projeo_utm_no_brasil.html

http://www.spatialreference.org/

A imagem a seguir, retirada do site do LAPIG (Laboratório de Processamento de Imagens e Georreferenciamento), explica basicamente como funciona o sistema de projeção UTM que utilizaremos no curso.



Já o site Spatial Reference nos fornecerá os CRS que precisaremos para modificar nossos arquivos.

In [1]:
import geopandas as gpd


In [2]:
rj = gpd.read_file('Mapas/RJ-MUNIC/RJ-MUNIC.shp')
geo_dados = gpd.read_file('Mapas/RJ-DATASET/DATASET.shp')

In [3]:
rj

,ID,CD_GEOCODM,NM_MUNICIP,geometry
0,1535,3304557,RIO DE JANEIRO,"MULTIPOLYGON (((-43.12376 -22.80712, -43.12380..."


In [4]:
geo_dados

,Area,Bairro,Cidade,Endereco,Estado,Latitude,Longitude,Quartos,Suites,Vagas,Tipo,Transacao,Valor,geometry
0,138,Recreio dos Bandeirantes,Rio de Janeiro,Rua Vicente Scofano,RJ,-23.016455,-43.462946,3,1,3,Apartamento,Venda,1095000,POINT (-43.46295 -23.01646)
1,133,Freguesia (Jacarepaguá),Rio de Janeiro,Rua Potiguara,RJ,-22.943199,-43.336911,3,1,2,Apartamento,Venda,650000,POINT (-43.33691 -22.94320)
2,46,Pilares,Rio de Janeiro,Rua Professor Carneiro Felipe,RJ,-22.873849,-43.300517,1,0,0,Apartamento,Venda,150000,POINT (-43.30052 -22.87385)
3,65,Barra da Tijuca,Rio de Janeiro,Avenida Salvador Allende,RJ,-22.984787,-43.412703,2,1,1,Apartamento,Venda,430000,POINT (-43.41270 -22.98479)
4,85,Gávea,Rio de Janeiro,Rua Marquês de São Vicente,RJ,-14.235004,-51.925280,2,1,2,Apartamento,Venda,1800000,POINT (-51.92528 -14.23500)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,48,Campo Grande,Rio de Janeiro,Rua Josué de Barros,RJ,-22.878349,-43.550762,2,0,1,Apartamento,Venda,170000,POINT (-43.55076 -22.87835)
4996,75,Recreio dos Bandeirantes,Rio de Janeiro,Rua Le Corbusier,RJ,-23.024717,-43.487285,3,1,1,Apartamento,Venda,550000,POINT (-43.48729 -23.02472)
4997,32,Jardim Botânico,Rio de Janeiro,Rua Getúlio das Neves,RJ,-22.959433,-43.206180,1,0,0,Apartamento,Venda,580000,POINT (-43.20618 -22.95943)
4998,85,Grajaú,Rio de Janeiro,Rua Jerônimo de Lemos,RJ,-22.918773,-43.260492,3,0,0,Apartamento,Venda,325000,POINT (-43.26049 -22.91877)


In [5]:
# ver qual CRS foi utilizado em cada um desses conjuntos de dados

rj.crs

<Geographic 2D CRS: EPSG:4674>
Name: SIRGAS 2000
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: Latin America - Central America and South America - onshore and offshore. Brazil - onshore and offshore.
- bounds: (-122.19, -59.87, -25.28, 32.72)
Datum: Sistema de Referencia Geocentrico para las AmericaS 2000
- Ellipsoid: GRS 1980
- Prime Meridian: Greenwich

In [6]:
geo_dados.crs

<Geographic 2D CRS: GEOGCS["GCS_unknown",DATUM["WGS_1984",SPHEROID["WG ...>
Name: GCS_unknown
Axis Info [ellipsoidal]:
- lon[east]: Longitude (Degree)
- lat[north]: Latitude (Degree)
Area of Use:
- undefined
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Antes de prosseguirmos, vamos nos atentar a alguns detalhes importantes. Na projeção UTM, sempre precisamos passar a zona e o hemisfério com que queremos trabalhar. Sabemos que o Brasil está quase inteiramente localizado no hemisfério sul, e que a cidade do Rio de Janeiro está inteiramente situada na zona 23.

Sabendo disso, no site http://www.spatialreference.org/, clicaremos em "EPSG" (http://www.spatialreference.org/ref/epsg/) e em seguida utilizaremos a caixa de busca para procurar pelo termo "sirgas 2000", que é o tipo de projeção oficialmente utilizado no Brasil. Dentre as várias opções que apareceção na tela, escolheremos "EPSG:31983: SIRGAS 2000 / UTM zone 23S" - ou seja, a projeção que é referente à zona 23 do hemisfério sul.

Na página que se abrirá, além de diversos dados sobre o EPSG:31983, teremos um mapa delineando a área comportada por essa projeção.

![](mapaproj.png)

Ainda nessa página, se clicarmos em "Proj4", o navegador nos retornará a seguinte configuração:

```python
+proj=utm +zone=23 +south +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=m +no_defs
```

De volta ao nosso código, iremos sobrescrever os CRSs dos nosso arquivos com a função to_crs(), passando como parâmetro a configuração que copiamos do Spatial Reference, com exceção de uma única modificação: ao invés de passarmos m (metros) para units, usaremos km (quilômetros).

In [7]:
rj = rj.to_crs('+proj=utm +zone=23 +south +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=km +no_defs')
geo_dados = geo_dados.to_crs('+proj=utm +zone=23 +south +ellps=GRS80 +towgs84=0,0,0,0,0,0,0 +units=km +no_defs')

In [8]:
rj.to_file('Mapas/RJ-MUNIC/RJ-MUNIC.shp')
geo_dados.to_file('Mapas/RJ-DATASET/DATASET.shp')

In [9]:
rj2 = gpd.read_file('Mapas/RJ-MUNIC/RJ-MUNIC.shp')
geo_dados2 = gpd.read_file('Mapas/RJ-DATASET/DATASET.shp')

In [10]:
rj2

,ID,CD_GEOCODM,NM_MUNICIP,geometry
0,1535,3304557,RIO DE JANEIRO,"MULTIPOLYGON (((692.576 7476.609, 692.572 7476..."


In [11]:
geo_dados2

,Area,Bairro,Cidade,Endereco,Estado,Latitude,Longitude,Quartos,Suites,Vagas,Tipo,Transacao,Valor,geometry
0,138,Recreio dos Bandeirantes,Rio de Janeiro,Rua Vicente Scofano,RJ,-23.016455,-43.462946,3,1,3,Apartamento,Venda,1095000,POINT (657.514 7453.832)
1,133,Freguesia (Jacarepaguá),Rio de Janeiro,Rua Potiguara,RJ,-22.943199,-43.336911,3,1,2,Apartamento,Venda,650000,POINT (670.524 7461.803)
2,46,Pilares,Rio de Janeiro,Rua Professor Carneiro Felipe,RJ,-22.873849,-43.300517,1,0,0,Apartamento,Venda,150000,POINT (674.345 7469.440)
3,65,Barra da Tijuca,Rio de Janeiro,Avenida Salvador Allende,RJ,-22.984787,-43.412703,2,1,1,Apartamento,Venda,430000,POINT (662.701 7457.284)
4,85,Gávea,Rio de Janeiro,Rua Marquês de São Vicente,RJ,-14.235004,-51.925280,2,1,2,Apartamento,Venda,1800000,POINT (-248.715 8415.117)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,48,Campo Grande,Rio de Janeiro,Rua Josué de Barros,RJ,-22.878349,-43.550762,2,0,1,Apartamento,Venda,170000,POINT (648.664 7469.216)
4996,75,Recreio dos Bandeirantes,Rio de Janeiro,Rua Le Corbusier,RJ,-23.024717,-43.487285,3,1,1,Apartamento,Venda,550000,POINT (655.010 7452.944)
4997,32,Jardim Botânico,Rio de Janeiro,Rua Getúlio das Neves,RJ,-22.959433,-43.206180,1,0,0,Apartamento,Venda,580000,POINT (683.909 7459.848)
4998,85,Grajaú,Rio de Janeiro,Rua Jerônimo de Lemos,RJ,-22.918773,-43.260492,3,0,0,Apartamento,Venda,325000,POINT (678.393 7464.417)


Perceba que tivemos uma mudança na coluna "geometry", que passou a exibir valores em quilômetros. Isso nos permitirá, por exemplo, medir as distâncias entre pontos desse conjunto.

Essa é a primeira configuração necessária para efetivamente começarmos a trabalhar no nosso projeto. O próximo passo é selecionarmos somente as informações referentes ao município do Rio de Janeiro, eliminando aqueles que não nos interessam.